# Making a hybrid CAD CSG model

can be useful for bioshields ...

In [ ]:
import openmc
import cadquery as cq
from cad_to_dagmc import CadToDagmc
import dagmc_h5m_file_inspector as di
from pathlib import Path

# Setting the cross section path to the correct location in the docker image.
# If you are running this outside the docker image you will have to change this path to your local cross section path.
openmc.config['cross_sections'] = Path.home() / 'nuclear_data' / 'cross_sections.xml'

In [ ]:
mat1 = openmc.Material(name='first_wall')
mat1.add_element('Fe', 1.0)
mat1.set_density('g/cm3', 7.87)

In [ ]:
major_radius = 1000
minor_radii = 400
thickness = 10

assembly = cq.Assembly()
inner_torus = cq.Solid.makeTorus(major_radius, minor_radii)
outer_torus = cq.Solid.makeTorus(major_radius, minor_radii+thickness)
shell = outer_torus.cut(inner_torus)
assembly.add(shell, name=f"first_wall")
assembly

In [ ]:
dag_mesh = CadToDagmc()
dag_mesh.add_cadquery_object(
    cadquery_object=assembly,
    material_tags="assembly_names",  # Use names as tags
)
dag_mesh.export_dagmc_h5m_file(set_size={'first_wall': 50} )

In [ ]:
di.convert_h5m_to_vtkhdf(h5m_filename='dagmc.h5m', vtkhdf_filename= 'dagmc.vtkhdf')

In [ ]:
dag_universe = openmc.DAGMCUniverse(filename='dagmc.h5m', auto_geom_ids=True)

In [ ]:
surf_x1 = openmc.XPlane(x0=-1000.0, name='x1', boundary_type='vacuum')
surf_x2 = openmc.XPlane(x0=1000.0, name='x2', boundary_type='vacuum')
surf_y1 = openmc.YPlane(y0=-1000.0, name='y1', boundary_type='vacuum')
surf_y2 = openmc.YPlane(y0=1000.0, name='y2', boundary_type='vacuum')
surf_z1 = openmc.ZPlane(z0=-1000.0, name='z1', boundary_type='vacuum')
surf_z2 = openmc.ZPlane(z0=1000.0, name='z2', boundary_type='vacuum')

region_inner_bioshield = +surf_x1 & -surf_x2 & +surf_y1 & -surf_y2 & +surf_z1 & -surf_z2
inner_bioshield = openmc.Cell(name='inner bioshield', region=region_inner_bioshield, fill=dag_universe)


In [ ]:
geometry = openmc.Geometry([inner_bioshield])


In [ ]:
# initialises a new source object
my_source = openmc.IndependentSource()

# the distribution of radius is just a single value
radius = openmc.stats.Discrete([major_radius], [1])

# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])

# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359)

# this makes the ring source using the three distributions and a radius
my_source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))

# sets the direction to isotropic
my_source.angle = openmc.stats.Isotropic()

# sets the energy distribution to a Muir distribution neutrons
my_source.energy = openmc.stats.muir(e0=14080000.0, m_rat=5.0, kt=20000.0)



In [ ]:
settings = openmc.Settings(
    run_mode="fixed source",
    source =my_source,
    particles=1000,
    batches=5
)

In [ ]:

model = openmc.Model(geometry, [mat1], settings)
model.run()
